In [291]:
import requests
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
from datetime import datetime, date, timedelta
import seaborn as sns
import matplotlib.pyplot as plt


# headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 \
# (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36",
# "Accept-language":"ru,en;q=0.9,ru-RU;q=0.8,en-US;q=0.7",
# "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,\
# */*;q=0.8,application/signed-exchange;v=b3;q=0.9"}

headers = {
    'authority': 'ms-mt--api-web.spain.advgo.net',
    'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
    'accept': 'application/json, text/plain, */*',
    'x-adevinta-channel': 'web-desktop',
    'x-schibsted-tenant': 'coches',
    'sec-ch-ua-mobile': '?0',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36',
    'content-type': 'application/json;charset=UTF-8',
    'origin': 'https://www.coches.net',
    'sec-fetch-site': 'cross-site',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': 'https://www.coches.net/',
    'accept-language': 'en-US,en;q=0.9,es;q=0.8',
}

In [292]:
def resnonse_url(url):
    """receiving a response from the server"""
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        print('-- works --')
    elif response.status_code == 404:
        print('-- not found --')
    else:
        print("-- something is wrong --", response.status_code)
    soup = BeautifulSoup(response.content, "html.parser")
    return soup

In [450]:
url = 'https://eurovisionworld.com/odds/eurovision'
soup = resnonse_url(url)

-- works --


In [451]:
def write_to_file(place, percent, country, artist, song, video, current_date):
    with open('euro.txt', 'a') as file:
                file.write(str(place) + ',' + str(percent) + ',' +
                           str(country) + ',' + str(artist) + ',' +
                           str(song) + ',' + str(video) + ',' + str(current_date) + '\n')

In [452]:
scrolls = soup.find_all(class_='odds_div scroll_x')
name_of_comp = scrolls[0].contents[0].contents[0]
odds = scrolls[0].contents[0].contents[1]

In [464]:
current_date = date.today()
datas = pd.read_csv('euro.txt', sep=',')

In [465]:
def parse(odds, place, current_date):
        artist = 'none'
        song = 'none'
        video = 'none'
        
        line = odds.contents[i].contents[2].contents[0]
        
        percent = odds.contents[i].contents[3].text
        percent = percent[:len(percent)-1]
        if percent[0] == '<':
            percent = 0.5
            
        # Для стран с названием больше 2 слов и объявленной песней
        if len(line) > 4:
            country = line.contents[4].text
            art = line.contents[6].text.split()
            artist = []
            let = 0
            while art[let] != '-':
                artist.append(art[let])
                let += 1
            artist = ' '.join(map(str, artist))
            song = ' '.join(map(str, art[let + 1:]))
            video = 'realise'
                
        # Для стран с названием больше 2 слов без песни, или для обычных стран с песней
        elif len(line) == 4:
            if line.contents[3].text in ['United Kingdom', 'Czech Republic'] :
                country = line.contents[3].text
            else:
                art = line.contents[3].text.split()
                artist = []
                let = 0
                while art[let] != '-':
                    artist.append(art[let])
                    let += 1
                artist = ' '.join(map(str, artist))
                song = ' '.join(map(str, art[let + 1:]))
                country = line.contents[2][1:len(line.contents[2])-1]
                video = 'realise'
        
        # Для стран с известным артистом, но без песни
        elif len(line) == 3:
            art = line.contents[2].text.split()
            if art[0] == 'artist:' or art[0] == 'artists:':
                artist = ' '.join(map(str, art[1:]))
            elif art[0] == 'song:':
                song = ' '.join(map(str, art[1:]))
            country = line.contents[1][1:len(line.contents[1])-1]
        
        # Для стран, у которых еще ничего не известно
        else:
            country = line.contents[1][1:]
            
        write_to_file(place, percent, country, artist, song, video, current_date)   

In [466]:
if str(current_date) == datas['date'][len(datas['date'])-1]:
    print('-- already done --')
else:
    for i in range(len(odds)):
        parse(odds, i+1, current_date)
    print('-- done --')

-- done --


In [468]:
datas = pd.read_csv('euro.txt', sep=',')
# print(datas.head())

In [469]:
def plot_by_date(datas):
    sns.set_theme(style="whitegrid")

    fig, ax = plt.subplots(figsize=(20, 23))

    cols = []
    labels = []
    colors = ['#89f13f','#f13f89', '#3f89f1']
    for index, row in datas.iterrows():
        if row[5] == 'realise':
            cols.append(colors[0])
            labels.append('with song')
        elif row[3] != 'none':
            cols.append(colors[2])
            labels.append('with artist')
        else:
            cols.append(colors[1])
            labels.append('empty')

#     sns.set(font_scale=1)
    ax = sns.barplot(x=[100 for _ in range(len(datas))], y="country", data=datas,
                dodge=False, palette=cols, alpha = 0.1)

    ax = sns.barplot(x="percent", y="country", data=datas, hue=labels,
                palette=colors, dodge=False)

    for index, row in datas.iterrows():
        ax.text(row.percent +0.2, row.place-1+0.15, row.percent, color='black', fontsize=12)

    max_lim = datas.percent.iloc[0]
    ax.legend(ncol=1, loc="lower right", frameon=True)
    plt.xlabel("Winning chance, %",fontsize=20)
    plt.ylabel("Country",fontsize=20)
    plt.tick_params(axis='both', labelsize=17)
    ax.set(xlim=(0, max_lim+10))
    plt.title(datas['date'].iloc[0], fontsize=20)
#     sns.despine(left=True, bottom=True)
    plt.savefig(str(datas['date'].iloc[0]) + '.png', dpi=100)
    plt.close()

In [470]:
for dates in datas['date'].unique():
    plot_by_date(datas[datas['date'] == dates])

In [210]:
# import pandas_alive

# datas["date"] = pd.to_datetime(datas["date"])
# data1 = datas.pivot(index='date', columns="country", values="percent").fillna(0)
# data1.plot_animated("euro_odds.gif", title='Odds', dpi=250)

In [471]:
import bar_chart_race as bcr

def one_day(datas, iters):
    cols = []
    labels = []
    trying = {}
    colors = ['#89f13f','#f13f89', '#3f89f1']
    for index, row in datas.iterrows():
        if row[5] == 'realise':
            trying[row[2]] = colors[0]
            cols.append(colors[0])
            labels.append('with song')
        elif row[3] != 'none':
            trying[row[2]] = colors[2]
            cols.append(colors[2])
            labels.append('with artist')
        else:
            trying[row[2]] = colors[1]
            cols.append(colors[1])
            labels.append('empty')
    new = dict(sorted(trying.items()))

    datas["date"] = pd.to_datetime(datas["date"])
    data1 = datas.pivot(index='date', columns="country", values="percent").fillna(0)
    bcr.bar_chart_race(df = data1, 
        filename = 'odds' + str(iters) + '.gif', 
#         filter_column_colors = True, 
        period_label={'x': .99, 'y':.05, 'ha': 'right', 'color': 'black'},
        bar_label_size = 7, tick_label_size = 7,
        bar_size = 0.8,
        steps_per_period = 20,
        dpi = 100,
    #     bar_kwargs={'alpha':0.4, 'ec': 'black', 'lw': 2.5, },
        cmap = list(new.values()),
#         cmap = ['#89f13f','#3f89f1', '#3f89f1', '#3f89f1'],
#         n_bars = 37,
        figsize=(10, 7),
        title = "Odds")

In [472]:
day_row = 0
iters = 0
while day_row < len(datas):
#     datas2 = datas.loc[day_row:day_row+73].copy()
    one_day(datas.loc[day_row:day_row+36].copy(), iters)
    iters += 1
    one_day(datas.loc[day_row:day_row+73].copy(), iters)
    day_row += 37
    iters += 1


/home/kurdoyakova/.local/lib/python3.8/site-packages/bar_chart_race/_make_chart.py:286: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(self.df_values.columns)
/home/kurdoyakova/.local/lib/python3.8/site-packages/bar_chart_race/_make_chart.py:287: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([max_val] * len(ax.get_xticks()))
/home/kurdoyakova/.local/lib/python3.8/site-packages/bar_chart_race/_make_chart.py:286: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(self.df_values.columns)
/home/kurdoyakova/.local/lib/python3.8/site-packages/bar_chart_race/_make_chart.py:287: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([max_val] * len(ax.get_xticks()))
/home/kurdoyakova/.local/lib/python3.8/site-packages/bar_chart_race/_make_chart.py:286: UserWarning: FixedFormatter should only be used together wit

In [108]:
# from matplotlib import animation

# fig=plt.figure()

# for n, dates in enumerate(datas['date'].unique()):
#     datas2 = datas[datas['date'] == dates]
# #     n=100 #Number of frames
#     cols = []
#     labels = []
#     colors = ['#89f13f','#f13f89', '#3f89f1']
#     for index, row in datas2.iterrows():
#         if row[5] == 'realise':
#             cols.append(colors[0])
#             labels.append('with song')
#         elif row[3] != 'none':
#             cols.append(colors[2])
#             labels.append('with artist')
#         else:
#             cols.append(colors[1])
#             labels.append('empty')
            
#     x = datas2['percent']
#     y = datas2['country']
#     barcollection = plt.barh(y, x, color = cols)
#     plt.barh(y, x, color = cols)
#     plt.gca().invert_yaxis()
#     plt.show()

# #     print(barcollection)
# def animate(i):
#     y=datas2['country']
#     for i, b in enumerate(barcollection):
#         b.set_height(y[i])

# anim=animation.FuncAnimation(fig,
#                              animate,
#                              repeat=False,
#                              blit=False,
#                              frames=datas['date'],
#                              interval=30)

# # anim.save('mymovie.mp4',writer=animation.FFMpegWriter(fps=10))
# # anim.save('first.gif',writer='imagemagick',fps=30)
# anim.save('first')

# for i in range(len(barcollection)):
#     barcollection[i].set_height(y[i])